<a href="https://colab.research.google.com/github/STASYA00/IAAC2024_tutorials/blob/main/quickstarts/06_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
df = pd.read_pickle('segmentation_data.pkl')

In [19]:
df.head()

,ImageId,Lat,Long,imagePath,road,sidewalk,building,wall,fence,pole,...,terrain,sky,person,rider,car,truck,bus,train,motorcycle,bicycle
0,0,2.218998,41.450931,/content/drive/MyDrive/streetview_images/Barce...,31.413269,3.634644,47.315979,0.000000,0.000000,0.057983,...,0.000000,14.123535,3.378296,0.061035,0.015259,0.000000,0.0,0.000000,0.000000,0.0
1,1,2.199195,41.405760,/content/drive/MyDrive/streetview_images/Barce...,29.707336,1.153564,43.658447,8.798218,3.369141,0.253296,...,0.012207,11.547852,0.003052,0.000000,0.201416,0.042725,0.0,0.000000,0.137329,0.0
2,2,2.099796,41.387263,/content/drive/MyDrive/streetview_images/Barce...,19.183350,4.736328,14.956665,4.281616,5.229187,1.174927,...,7.893372,40.925598,0.000000,0.000000,0.288391,0.000000,0.0,0.006104,0.000000,0.0
3,3,2.142156,41.425825,/content/drive/MyDrive/streetview_images/Barce...,43.208313,3.437805,16.857910,0.000000,0.000000,0.000000,...,0.569153,14.407349,0.000000,0.000000,0.170898,0.000000,0.0,0.000000,0.000000,0.0
4,4,2.208730,41.421822,/content/drive/MyDrive/streetview_images/Barce...,40.287781,0.737000,12.271118,0.000000,0.039673,0.035095,...,1.792908,18.214417,0.064087,0.000000,4.724121,0.000000,0.0,0.000000,2.938843,0.0


In [20]:
df.columns

Index(['ImageId', 'Lat', 'Long', 'imagePath', 'road', 'sidewalk', 'building',
       'wall', 'fence', 'pole', 'traffic light', 'traffic sign', 'vegetation',
       'terrain', 'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train',
       'motorcycle', 'bicycle'],
      dtype='object')

In [21]:
from sklearn.cluster import KMeans
import numpy as np

X = df.drop(columns=["ImageId", "Lat", "Long", "imagePath"])
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X)
kmeans.labels_

array([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int32)

In [22]:
# folium is a mapping libary
!pip install folium
import folium

# for colors
import branca

In [23]:
df["kmeans_2"] = kmeans.labels_

In [24]:
map_center = [df['Long'].mean(), df['Lat'].mean()]

In [25]:
m = folium.Map(location=map_center, zoom_start=11)

color_column="kmeans_2"

#  color mapping (list with colors)
colormap = branca.colormap.linear.YlOrRd_09.scale(df[color_column].min(), df[color_column].max())

# Add a marker for each row in the DataFrame
for _, row in df.iterrows():
    # get a color
    color = colormap(row[color_column])

    folium.CircleMarker(
        location=[row['Long'], row['Lat']],
        radius=6,
        color= None,# adjust the radius for the size of the dot         # outline color
        fill=True,
        fill_color= color, # fill color
        fill_opacity=0.7,
        popup=f"ImageId: {row['ImageId']}"
    ).add_to(m)

# Display the map
m

In [26]:
kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto").fit(X)
kmeans.labels_

array([1, 1, 0, 4, 4, 3, 1, 2, 1, 0, 0, 1, 3, 3, 2, 0, 1, 3, 3, 3, 3, 0,
       3, 1, 2, 3, 3, 2, 3, 3, 2, 3, 3, 2, 4, 3, 3, 3, 4, 3, 3, 4, 1, 3,
       0, 1, 4, 4, 4, 4, 3, 2, 2, 2, 2, 3, 3, 2, 0, 0, 3, 3, 1, 4, 3, 4,
       1, 3, 4, 2, 2, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3,
       3, 2, 3, 2, 1, 1, 4, 2, 2, 4, 0, 4, 2, 1, 1, 3, 1, 0, 2, 4, 3, 2,
       3, 2, 1, 4, 3, 3, 2, 3, 2, 0, 4, 3, 4, 2, 3, 3, 2, 2, 2, 1, 3, 2,
       4, 4, 4, 4, 2, 1, 1, 1, 0, 4, 3, 3, 0, 1, 2, 2, 1, 3, 1, 4, 1, 3,
       0, 1, 4, 3, 4, 1, 4, 0, 1, 1, 1, 2, 0, 3, 4, 3, 3, 3, 2, 1, 2, 4,
       3, 3, 4, 1, 0, 4, 3, 2, 3, 1, 3, 1, 3, 1, 2, 0, 3, 1, 1, 0, 3, 2,
       3, 1, 0, 2, 3, 1, 0, 1, 2, 3, 3, 3, 1, 0, 3, 2, 3, 0, 3, 2, 4],
      dtype=int32)

In [27]:
df["kmeans_5"] = kmeans.labels_
m = folium.Map(location=map_center, zoom_start=11)

color_column="kmeans_5"

#  color mapping (list with colors)
colormap = branca.colormap.linear.YlOrRd_09.scale(df[color_column].min(), df[color_column].max())

# Add a marker for each row in the DataFrame
for _, row in df.iterrows():
    # get a color
    color = colormap(row[color_column])

    folium.CircleMarker(
        location=[row['Long'], row['Lat']],
        radius=6,
        color= None,# adjust the radius for the size of the dot         # outline color
        fill=True,
        fill_color= color, # fill color
        fill_opacity=0.7,
        popup=f"ImageId: {row['ImageId']}"
    ).add_to(m)

# Display the map
m

In [28]:
kmeans.cluster_centers_

array([[1.31221605e+01, 7.15179443e+00, 7.09792427e+00, 4.00695801e+00,
        5.02279530e+00, 4.92195461e-01, 3.98055367e-04, 8.02744990e-02,
        1.20533819e+01, 9.63798191e+00, 3.71100384e+01, 4.36268682e-01,
        1.26780634e-01, 2.63837731e+00, 2.69815196e-01, 2.66697096e-02,
        3.13004203e-01, 3.62893809e-01, 5.02876613e-02],
       [2.34453423e+01, 5.26069819e+00, 5.12731508e+01, 4.41724201e-01,
        1.42517090e+00, 2.21181470e-01, 8.16167787e-04, 5.56768373e-02,
        2.39435240e+00, 1.38393668e-01, 6.88920132e+00, 2.60552695e+00,
        2.70399936e-02, 4.00025124e+00, 2.43147029e-01, 6.20287518e-02,
        6.45056436e-01, 6.85368028e-01, 1.85873342e-01],
       [2.68548300e+01, 4.21540016e+00, 4.36699446e+00, 1.54677990e+00,
        1.91859755e+00, 2.63870594e-01, 3.54855560e-03, 6.99420308e-02,
        4.01758593e+01, 2.61762752e+00, 1.37571468e+01, 3.95167151e-01,
        9.79401344e-03, 3.19487106e+00, 1.73879224e-02, 1.81402162e-01,
        1.38606582e-01

In [44]:
from sklearn.cluster import DBSCAN
import numpy as np
X = df[["Lat", "Long"]]
clustering = DBSCAN(eps=0.01, min_samples=2).fit(X)
df["density"] = clustering.labels_

In [45]:
df["density"].max()

15

In [46]:
m = folium.Map(location=map_center, zoom_start=11)

color_column="density"

#  color mapping (list with colors)
colormap = branca.colormap.linear.YlOrRd_09.scale(df[color_column].min(), df[color_column].max())

# Add a marker for each row in the DataFrame
for _, row in df.iterrows():
    # get a color
    color = colormap(row[color_column])

    folium.CircleMarker(
        location=[row['Long'], row['Lat']],
        radius=6,
        color= None,# adjust the radius for the size of the dot         # outline color
        fill=True,
        fill_color= color, # fill color
        fill_opacity=0.7,
        popup=f"ImageId: {row['ImageId']}"
    ).add_to(m)

# Display the map
m